# 전체 classification 결과를 환자 단위, 파트 별 단위로 묶어서 결과 만드는 코드

In [14]:
import os
import numpy as np
import pandas

## ENV SETTING

In [15]:
label_type = '5classes'
learning_rate = '5e-5'

# true label env
true_dataset_root = 'E:/Thesis_research/Database/Medical/Dental_directory_dataset'
true_lbl_dir = os.path.join(true_dataset_root, 'ClassificationClass',label_type)

# prediction env
pred_root = f'E:/Thesis_research/results_materials/Dental/raw_prediction_results/{learning_rate}'

In [16]:
def convert_5_to_3_confmat(five_group_confmat):
    three_group_confmat = np.zeros((3,3))
    three_group_confmat[0,0] = five_group_confmat[0,0]
    three_group_confmat[0,-1] = five_group_confmat[0,-1]
    three_group_confmat[-1,0] = five_group_confmat[-1,0]
    three_group_confmat[-1,-1] = five_group_confmat[-1,-1]
    
    three_group_confmat[0,1] = np.sum(five_group_confmat[0, 1:4])
    three_group_confmat[1,0] = np.sum(five_group_confmat[1:4, 0])
    three_group_confmat[1,-1] = np.sum(five_group_confmat[1:4, -1])
    three_group_confmat[-1,1] = np.sum(five_group_confmat[-1, 1:4])
    
    
    three_group_confmat[1,1] = np.sum(five_group_confmat[1:4, 1:4])
    
    return three_group_confmat
    

## PREDICTION SETTING AND VOTING

* 각 네트워크 별로 4개의 part에 대한 prediction 중 unique 병록번호에 해당하는 prediction들을 모아서 voting해서 true와 비교!

In [17]:
from collections import Counter
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

part_list = [16, 26, 36, 46]
exp_dir = os.path.join(pred_root, label_type)
fold_name_list = sorted(os.listdir(exp_dir))

patient_wise_overall_acc_lst = []
part_wise_overall_acc_dic = {}
for i_part in part_list:
    part_wise_overall_acc_dic[i_part] = []

confusion_matrix_metric_five_group_tot_list = []
confusion_matrix_metric_three_group_tot_list = []
confusion_matrix_metric_partwise_tot_dict = {}
for i_part in part_list:
    confusion_matrix_metric_partwise_tot_dict[i_part] = []
    
    
for i_fold_iter, i_fold_name in enumerate(fold_name_list):
    print()
    print('Current: ', i_fold_name)
    
    # ## TRUE LABEL SETTING

    # * 병록번호 - true label 페어 만들자
    true_imageset_path = os.path.join(true_dataset_root,'ImageSets','Classification','eval' + str(i_fold_iter+1) + '.txt')
    
    with open(true_imageset_path, 'r') as f:
         eval_img_list = f.read().split('\n')

    person_num_list =[]
    true_lbl_list = []
    for i_eval_img in eval_img_list:
        if i_eval_img == '':
            continue
        eval_img_info = i_eval_img.split('_')
        age_person_num = eval_img_info[0] + '_' + eval_img_info[1] # e.g. '20_2392392' because there are three miss labeled images file name
        if len(eval_img_info)>1: # skip blank line
            person_num_list.append(age_person_num)
    person_num_unique_list, unique_idx = np.unique(np.array(person_num_list), return_index=True)
    
    pred_by_part = {}
    true_by_part = {}
    for i_part  in part_list:
        pred_by_part[i_part] = []
        true_by_part[i_part] = []
    
    person_num_perdiction_all_list = []
    true_lbl_unique = []
    for i_iter, i_person_num_unique in enumerate(person_num_unique_list):
        pred_dir = os.path.join(exp_dir,i_fold_name, 'eval_result_resnet152_cls_best_model', 'prediction_class')
        pred_result_list = sorted(os.listdir(pred_dir))
        pred_result_person_num = [s for s in pred_result_list if i_person_num_unique in s]
             
        # 하나라도 파트 없으면 skip!!!!!
        if len(pred_result_person_num) < 4:
            print('Each person must have four teeth parts')
            continue
                   
        # true label setting
        true_lbl = 0
        for i, i_pred in enumerate(pred_result_person_num):
            true_lbl_path = os.path.join(true_lbl_dir, i_pred)
            with open(true_lbl_path,'r') as f:
                lbl = int(f.read())
            if i==0:
                true_lbl = lbl
            else:
                if true_lbl != lbl: # check all patients label is the same each other
                    raise AssertionError
                else:
                    true_lbl = lbl
        true_lbl_unique.append(true_lbl)
                
        person_num_prediction = []
        
        for i_pred in pred_result_person_num:
            pred_txt_nameOnly = os.path.splitext(i_pred)[0]
            pred_name_info = pred_txt_nameOnly.split('_')
            part_num = int(pred_name_info[-1])

            pred_result_path = os.path.join(pred_dir, i_pred)
            with open(pred_result_path, 'r') as f:
                pred_lbl = int(f.read())
            person_num_prediction.append(pred_lbl)
            pred_by_part[part_num].append(pred_lbl)
            true_by_part[part_num].append(true_lbl)
        
        person_num_perdiction_all_list.append(person_num_prediction)
    
    network_final_pred_list = []
    for i_person_num_pred in person_num_perdiction_all_list:
        most_common_pred, num_most_common_pred = Counter(i_person_num_pred).most_common(1)[0] # 4, 6 times
        network_final_pred_list.append(most_common_pred)
    
    print('===FIVE GROUPS===')
    confusion_matrix_metric = confusion_matrix(true_lbl_unique, network_final_pred_list)
    confusion_matrix_metric_five_group_tot_list.append(confusion_matrix_metric)
    print('Confusion matrix: ')
    print(confusion_matrix_metric)
    overall_acc_metric = accuracy_score(true_lbl_unique, network_final_pred_list)
    print('Overall accuracy = ', overall_acc_metric)
    
    print('===THREE GROUPS===')
    confusion_matrix_metric_three_group = convert_5_to_3_confmat(confusion_matrix_metric)
    print('Confusion matrix: ')
    print(confusion_matrix_metric_three_group)
    confusion_matrix_metric_three_group_tot_list.append(confusion_matrix_metric_three_group)
    overall_acc_metric_three_group = np.sum(np.diag(confusion_matrix_metric_three_group)) / np.sum(confusion_matrix_metric_three_group)
    print('Overall accuracy = ', overall_acc_metric_three_group)
    patient_wise_overall_acc_lst.append(overall_acc_metric_three_group)
    
    ## save as excel
    index =['True'+str(i) for i in range(1, confusion_matrix_metric.shape[0]+1)]
    columns = ['Pred'+str(i) for i in range(1, confusion_matrix_metric.shape[0]+1)]
    df_total = pandas.DataFrame(confusion_matrix_metric, index=index, columns = columns)
    save_excel_path = os.path.join(exp_dir, i_fold_name, 'conf_mat_total.xlsx')
    df_total.to_excel(save_excel_path)
    
    print('======== part by part metric ==========')
    for key in pred_by_part:
        print('PART NUM: ', key)
        confusion_matrix_metric = confusion_matrix(true_by_part[key], pred_by_part[key])
        print('===FIVE GROUPS===')
        print('Confusion matrix: ')
        print(confusion_matrix_metric)
        overall_acc_metric = accuracy_score(true_by_part[key], pred_by_part[key])
        print('Overall accuracy = ', overall_acc_metric)
            
        print('===THREE GROUPS===')
        confusion_matrix_metric_three_group = convert_5_to_3_confmat(confusion_matrix_metric)
        print('Confusion matrix: ')
        print(confusion_matrix_metric_three_group)
        overall_acc_metric_three_group = np.sum(np.diag(confusion_matrix_metric_three_group)) / np.sum(confusion_matrix_metric_three_group)
        print('Overall accuracy = ', overall_acc_metric_three_group)
        part_wise_overall_acc_dic[key].append(overall_acc_metric_three_group)

        index =['True'+str(i) for i in range(1, confusion_matrix_metric.shape[0]+1)]
        columns = ['Pred'+str(i) for i in range(1, confusion_matrix_metric.shape[0]+1)]
        df_part = pandas.DataFrame(confusion_matrix_metric, index=index, columns = columns)
        save_excel_path = os.path.join(exp_dir, i_fold_name, 'conf_mat_part' + str(key) +'.xlsx')
        df_total.to_excel(save_excel_path)

confmat_save_path = os.path.join(exp_dir, '5cls_conf_mat_tot.npy')
np.save(confmat_save_path, confusion_matrix_metric_five_group_tot_list)

confmat_save_path = os.path.join(exp_dir, '5cls_conf_mat_tot(regroup).npy')
np.save(confmat_save_path, confusion_matrix_metric_three_group_tot_list)

        


Current:  resnet152-TL_aug-5classes-fold0
===FIVE GROUPS===
Confusion matrix: 
[[ 73   6   0   0   0]
 [  1 103   6   1   0]
 [  4  19  20   6   5]
 [  0   8  12   7   9]
 [  0   1   1   9  36]]
Overall accuracy =  0.7308868501529052
===THREE GROUPS===
Confusion matrix: 
[[ 73.   6.   0.]
 [  5. 182.  14.]
 [  0.  11.  36.]]
Overall accuracy =  0.8899082568807339
======== part by part metric ==========
PART NUM:  16
===FIVE GROUPS===
Confusion matrix: 
[[72  6  1  0  0]
 [ 5 96  8  2  0]
 [ 3 15 21  9  6]
 [ 1  9  9  9  8]
 [ 0  2  1  8 36]]
Overall accuracy =  0.7155963302752294
===THREE GROUPS===
Confusion matrix: 
[[ 72.   7.   0.]
 [  9. 178.  14.]
 [  0.  11.  36.]]
Overall accuracy =  0.8746177370030581
PART NUM:  26
===FIVE GROUPS===
Confusion matrix: 
[[ 73   6   0   0   0]
 [  5 101   3   1   1]
 [  5  24  11   8   6]
 [  0   8   9  11   8]
 [  0   0   1  10  36]]
Overall accuracy =  0.709480122324159
===THREE GROUPS===
Confusion matrix: 
[[ 73.   6.   0.]
 [ 10. 176.  15.]
 

# Patient wise cv 결과 정리 (three group)

In [18]:
print('Confusion matrix: ')
confusion_matrix_metric_tot = np.array(confusion_matrix_metric_three_group_tot_list)
confusion_matrix_metric_avg = np.mean(confusion_matrix_metric_tot, axis = 0)
print(confusion_matrix_metric_avg)
print()
print('Overall Accuracy: ')
patient_wise_avg_acc = np.mean(patient_wise_overall_acc_lst)
patient_wise_std= np.std(patient_wise_overall_acc_lst)
print('acc: ',patient_wise_avg_acc)
print('std: ', patient_wise_std)
print()
print('Group-wise accuracy: ')
group_wise_acc_dict={}
for i_group in range(confusion_matrix_metric_tot.shape[1]):
    group_wise_acc_dict[i_group] = []
    for i_fold in range(confusion_matrix_metric_tot.shape[0]):
        confusion_matrix_cur = confusion_matrix_metric_tot[i_fold]
        group_wise_acc = confusion_matrix_cur[i_group, i_group] / np.sum(confusion_matrix_cur[i_group, :])
        group_wise_acc_dict[i_group].append(group_wise_acc)
        
    group_wise_acc_mean = np.mean(group_wise_acc_dict[i_group])
    group_wise_acc_std = np.std(group_wise_acc_dict[i_group])
    print('Age group ' + str(i_group+1))
    print('acc: ',group_wise_acc_mean)
    print('std: ',group_wise_acc_std)
    print()


Confusion matrix: 
[[ 72.4   6.8   0. ]
 [  3.6 181.   13.8]
 [  0.    7.4  31.8]]

Overall Accuracy: 
acc:  0.9004188646310869
std:  0.01087105211996819

Group-wise accuracy: 
Age group 1
acc:  0.9144604024667317
std:  0.0396187094862616

Age group 2
acc:  0.9124289032430944
std:  0.012071177200491194

Age group 3
acc:  0.8094272291121479
std:  0.05147410601422207



# Part wise cv 결과 정리

In [20]:
for i_part in part_list:
    print('===============')
    print('Part', i_part)

    print('Overall Accuracy: ')
    part_wise_avg_acc = np.mean(part_wise_overall_acc_dic[i_part])
    part_wise_std_error= np.std(part_wise_overall_acc_dic[i_part]) / np.sqrt(len(part_wise_overall_acc_dic[i_part]))
    print('acc: ', part_wise_avg_acc)
    print('std_error: ', part_wise_std_error)
    print()



Part 16
Overall Accuracy: 
acc:  0.8786771365804198
std_error:  0.007233779760827034

Part 26
Overall Accuracy: 
acc:  0.8740235591949223
std_error:  0.005774127829640781

Part 36
Overall Accuracy: 
acc:  0.8835911071622162
std_error:  0.007364316690240947

Part 46
Overall Accuracy: 
acc:  0.881512832052856
std_error:  0.004354418974369726



## Patient-wise, part-wise 비교

In [21]:
print('==== Patient-wise overall acc ====')
patient_wise_avg_acc = np.mean(patient_wise_overall_acc_lst)
patient_wise_std_error= np.std(patient_wise_overall_acc_lst) / np.sqrt(len(patient_wise_overall_acc_lst))
print('avg: ',patient_wise_avg_acc * 100)
print('std_error: ', patient_wise_std_error*100)

print('==== Part-wise overall acc ====')
for i_part in part_list:
    print('Part', i_part)
    part_wise_avg_acc = np.mean(part_wise_overall_acc_dic[i_part])
    part_wise_std_error= np.std(part_wise_overall_acc_dic[i_part]) / np.sqrt(len(part_wise_overall_acc_dic[i_part]))
    print('avg: ', part_wise_avg_acc*100)
    print('std_error: ', part_wise_std_error*100)
    print()

==== Patient-wise overall acc ====
avg:  90.04188646310868
std_error:  0.4861682305438414
==== Part-wise overall acc ====
Part 16
avg:  87.86771365804198
std_error:  0.7233779760827034

Part 26
avg:  87.40235591949222
std_error:  0.577412782964078

Part 36
avg:  88.35911071622162
std_error:  0.7364316690240947

Part 46
avg:  88.15128320528561
std_error:  0.4354418974369726



In [22]:
## save data for using at t-test in 3cls+5cls voting
np.save(os.path.join(exp_dir, '5cls_patient_wise_acc_lst.npy'), patient_wise_overall_acc_lst)
for i_part in part_list:
    out_file_name = os.path.join(exp_dir, '5cls_part'+str(i_part)+'_acc_lst.npy')
    np.save(out_file_name, part_wise_overall_acc_dic[i_part])


In [23]:
from scipy.stats import ttest_ind
for i_part in part_list:
    print('Patient-wise acc vs part ' + str(i_part) + ' acc')
    ttest,pval = ttest_ind(patient_wise_overall_acc_lst,part_wise_overall_acc_dic[i_part])
    print("p-value", pval)
    print()

Patient-wise acc vs part 16 acc
p-value 0.05619258679166393

Patient-wise acc vs part 26 acc
p-value 0.014064268077519999

Patient-wise acc vs part 36 acc
p-value 0.12646740376844912

Patient-wise acc vs part 46 acc
p-value 0.03206653037540319



In [24]:
from scipy.stats import ttest_ind
for i_part in part_list:
    print('Patient-wise acc vs part ' + str(i_part) + ' acc')
    ttest,pval = ttest_ind(patient_wise_overall_acc_lst,part_wise_overall_acc_dic[i_part], equal_var=False)
    print("p-value",pval)
    print()

Patient-wise acc vs part 16 acc
p-value 0.06085742533823349

Patient-wise acc vs part 26 acc
p-value 0.014577967080453986

Patient-wise acc vs part 36 acc
p-value 0.1322748838941613

Patient-wise acc vs part 46 acc
p-value 0.032385068558745095

